# NYC 311 Service Requests - Data Gathering
### Danielle Medellin [GitHub](https://github.com/dmedellin2)

## Problem Statement

According to the NYC311 online portal, "a Service Request is your request for the City to provide you with assistance, perform an inspection, or address a problem. NYC311 can accept Service Requests for a wide range of issues, including over 500 complaint types. Service Requests can help you get a pothole fixed in your neighborhood, the heat turned on in your apartment, or a refund for an overpaid parking ticket." Due to the fact that there are so many different reasons for submitting service requests to 311, all with a wide range of urgency and importance, it can be difficult to gauge how long it will take for a request to actually get resolved.


The NYC 311 receives millions of requests a year. In fact, in 2018 they received 3,154,013 service requests! [(Source)](https://www1.nyc.gov/311/311-sets-new-record-in-2018.page)

A citizen can submit a 311 request in various ways including by phone, online, or using mobile. If they do so with an email or a phone number, they will get a confirmation message which includes the details of the request. What is not included in this confirmation, is expected wait time until the request will be resolved and closed. Obviously, there are many factors that can affect one's wait time, but given the amount of 311 requests received, it should be possible to find trends in the wait times of these requests and give citizens, at the minimum, an estimate of how long they can expect to wait until their request is resolved. 

Our goal is to build a regression model that will predict the wait time a citizen can expect given the factors of their request. Our model's success will be measured with root-mean squared error (RMSE). 

Additionally, we hope to build an applet protoype that mimics what inputting a service request might look like, and in addition to just getting a description of the request, the app will also return an estimated wait time for the request to be closed.

## Executive Summary

Data was collected using the [NYC Open Data API](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9). This database is updated daily with service requests from NYC 311. It currently holds 23 million observations each with 41 columns of information. We built a series of functions to collect recent data from this source through the API. 

After a few test runs of data collection, it was initially observed that there were many factors of what could affect the time it takes to resolve a service request. Looking at the most recent requests submitted, the overwheming majority of requests were sent to the New York City Police Department (NYPD). The NYPD responds to requests regarding noise, parking violations, trespassing, etc. The breadth of requests covered by the NYPD was considered varied enough to explore strictly requests from this one agency. 

Data was collected starting at Feb 1, 2020 and working backwards. Only observations that were sent to the NYPD and has a status of closed were considered. After discovering that 50,000 observations covered roughly a two week time span, we decided our goal was to collect about 6 months worth of data. This led us to collect the 600,000 most recent observations from our start date (Feb 1, 2020). Within our data collection we feature engineered our target variable of `waittime`, which was the difference in time from the `created_date` of a request to its `closed_date`. 

This set of 600,000 observations was split into a training and holdout set of 540,000 and 60,000 observations respectively. 

Once data was collected we performed data cleaning which involved giving each variable its appropriate data type, handling null values, and taking a closer look at our target variable. In our data exploration it was found that there were some negative wait times. This was unexpected, as it does not make sense for a request to take _negative_ time to complete. There were only 16 observations in our training set that had this issue, so they were dropped from the set. It was also found that the majority of requests were resolved in under one day. Despite this, there were some observations that had longer wait times spanning from 8 to 20 to 200 days. We decided that there should be a cut off for "reasonable" wait times, expecting that any request that takes over a month to resolve is not the norm. For this reason, we only looked at requests that had a wait time of 31 days or less. After this our training set had 539,312 observations. 

Further exploratory data analysis was performed as we explored how each of the key features affected wait time. The borough of the request was shown to have an affect on wait time as requests in the Bronx had the highest average wait time and those from Manhattan had the lowest. Of the top 10 most frequent complaint types, most had similar wait time averages. The complaint of _**Abandoned Vehicle**_ had the highest wait time average.

Before putting our chosen features into a model, we used $k$-modes clustering to potentially find similarities between the observations. We created 5 clusters and used these cluster categories as an additional feature. 

We tried many regression models including linear regression, decision trees, random forest, Gamma regression (GLM), and an AdaBoost model. Unfortunately, none of these models performed much better than our baseline model. 

We also attempted a time series model of order (0, 0, 1), or a MA(1) model using the `created_date` feature. This model looked at the average wait time of each day in our data set which decreased the sample size from almost 540,000 observations to just 249 observations. This model performed much better than the previous regression models stated above, but could not encapsulate the specifics of each observation as we desired. 

Moving forward, we used the GLM Gamma regression model to make predictions and evaluate our model. When looking at the coefficients of this model, it was found that the borough the request came from had the most variation in affect on wait time. Each cluster had similar affects on the target variable.

Finally, an app was created so that a user could simply visit the website, input the details of their request, and immediately receive an estimation of how long it will take for their request to be resolved. 

### Loading Libraries

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import requests
from sodapy import Socrata
import json
import matplotlib.pyplot as plt
import seaborn as sns
import os

In order to replicate the data collection in this notebook, the user needs to acquire an app token by following the steps laid out [here](https://dev.socrata.com/foundry/data.cityofnewyork.us/erm2-nwe9). 

The user is free to plug their app token directly into this notebook or create an `env.json` file similar to this project. 

In [2]:
ENV = pd.read_json("../env.json", typ = "series")
API_KEY = ENV["APIKEY"]
API_SECRET = ENV["APISECRET"]
APP_TOKEN = ENV["APPTOKEN"]
APP_SECRET = ENV["APPSECRET"]

### Data Collection

In [12]:
# function to convert datetimes to all days
def conv_to_days(waittime): # must be a timedelta type 
    """
    Converts datetime object in units of days
    """
    try:
        days = abs(waittime.components[0])
        hours = waittime.components[1] / 24
        mins = waittime.components[2] / 60 / 24
        secs = waittime.components[3] / 60 / 60 / 24
        if waittime.components[0] < 0:
            return (days + hours + mins + secs) * -1 # returns waittime in days with original sign
        else:
            return days + hours + mins + secs
    # for NaT types
    except:
        pass


def get_311_data(limit=2000, app_token=None, date_max='2020-02-01T00:00:00'): # takes limit, and app_token, not necessary
    
    """
    Collects 311 service request data and returns a data frame. 
    
    Arguments:
    - limit: how much data you want to collect in each sweep. Default set to 2000 (MAX). Will collect limit x 25 rows of data.
    - app_token: not necessary, but will help with the collection process.
    - date_max: set the most recent date you want to collect from in the form 'YYYY-MM-DDTHH:MM:SS'
    """
    
    # columns we want to grab from dataframe
    SUBFIELDS = ['unique_key', 'created_date','closed_date', 'agency', 'agency_name', 'complaint_type',
       'descriptor', 'location_type', 'status', 'community_board', 'borough',
       'open_data_channel_type', 'park_facility_name', 'park_borough',
       'incident_zip', 'incident_address', 'street_name', 'cross_street_1',
       'cross_street_2', 'intersection_street_1', 'intersection_street_2',
       'city', 'landmark', 'bbl', 'x_coordinate_state_plane',
       'y_coordinate_state_plane', 'latitude', 'longitude', 'location', 'resolution_description',
       'resolution_action_updated_date']
    
    # establish client source
    client = Socrata("data.cityofnewyork.us", app_token, timeout=300)
    
    data = []
    
    for offs in range(0, 48001, 2000): # will cycle through 50000 entries when limit is 2000
        # Collect results offset by 2000, returned as JSON from API / converted to Python list of
        # dictionaries by sodapy.
        results = client.get("erm2-nwe9",limit=limit,
                             offset=offs,
                             # looks for only NYPD agencies, closed requests, before Feb 2020
                             where=f"agency = 'NYPD' and status = 'Closed' and created_date < '{date_max}'",
                             order="created_date desc") # takes from most recent

        # Convert to pandas DataFrame
        results_df = pd.DataFrame.from_records(results, columns = SUBFIELDS)
        
        # add df to list 
        data.append(results_df)
        time.sleep(3)
    
    full = pd.concat(data)
    full.reset_index(inplace=True)
    
    # cleaning & engineering features 
    full['created_date'] = pd.to_datetime(full['created_date']) # change type to datetime
    full['closed_date'] = pd.to_datetime(full['closed_date']) # change type to datetime
    full['waittime'] = full['closed_date'] - full['created_date'] # find wait time
    full['waittime'] = full['waittime'].map(conv_to_days) # convert wait time to days
    
    return full

The above function will use the NYC Open Data API to pull the most recent 50,000 entries of requests that fall under the NYPD agency, are classified as closed, and occurred before Feb 1 2020. The columns of data being pulled are specific in `SUBFIELDS`. The function will return a data frame with appropriate columns including an engineered feature, `waittime`. 

In [5]:
def get_earliest_date(df): # will get earliest date from data frame so we know where to start for next one
    return str(df['created_date'][49999]).replace(" ", "T")

In [9]:
def get_all_data(app_token):
    
    dfs = []
    
    df1 = get_311_data(app_token = app_token)
    df_last = get_earliest_date(df1)
    dfs.append(df1)
    
    for _ in range(2,13): # repeat pull 12 times 
        df = get_311_data(app_token = app_token, date_max = df_last)
        df_last = get_earliest_date(df)
        dfs.append(df)
    
    full_data = pd.concat(dfs, ignore_index = True)
    
    return full_data

The two functions above will allow the user to repeat the process of pulling data starting off where the last pull ended. We are repeating our original data pull function 12 times to acquire 600,000 observations, or roughly 6 months of data. 

In [13]:
# collecting data
sr_df = get_all_data(APP_TOKEN)

In [15]:
sr_df.drop(columns='index', inplace = True)

In [16]:
# checking data frame
sr_df.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,status,community_board,...,landmark,bbl,x_coordinate_state_plane,y_coordinate_state_plane,latitude,longitude,location,resolution_description,resolution_action_updated_date,waittime
0,45514479,2020-01-31 23:58:46,2020-02-01 11:05:48,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,Closed,06 BRONX,...,HOFFMAN STREET,2030540026,1015007,251021,40.85561595630976,-73.88881679868403,"{'latitude': '40.85561595630976', 'longitude':...",The Police Department responded to the complai...,2020-02-01T16:05:49.000,0.463218
1,45517361,2020-01-31 23:58:28,2020-02-01 02:57:57,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,NaN,Closed,11 MANHATTAN,...,EAST 100 STREET,1016280006,998065,226692,40.78888251589769,-73.95011034864612,"{'latitude': '40.78888251589769', 'longitude':...",The Police Department responded to the complai...,2020-02-01T07:58:02.000,0.124641
2,45516851,2020-01-31 23:57:05,2020-02-01 01:36:02,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,Closed,14 BROOKLYN,...,FOSTER AVENUE,3052200064,995846,171145,40.63642248430778,-73.95821942468432,"{'latitude': '40.63642248430778', 'longitude':...",The Police Department responded to the complai...,2020-02-01T06:36:05.000,0.068715
3,45515360,2020-01-31 23:56:48,2020-02-01 02:16:21,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,Closed,02 MANHATTAN,...,BROOME STREET,1004777502,982629,203186,40.72437519160271,-74.00584819694839,"{'latitude': '40.72437519160271', 'longitude':...",The Police Department responded to the complai...,2020-02-01T07:16:25.000,0.096910
4,45518349,2020-01-31 23:56:42,2020-02-01 08:52:48,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,Closed,09 BRONX,...,TAYLOR AVENUE,2039000033,1021265,244157,40.8367522977154,-73.86623273386101,"{'latitude': '40.8367522977154', 'longitude': ...",The Police Department responded to the complai...,2020-02-01T13:52:50.000,0.372292


In [17]:
sr_df.shape

(600000, 32)

We successfully pulled 600,000 observations with 32 features.

In [22]:
# write data to csv 
if os.path.exists('../data') == True:
    pass
else:
    os.mkdir('../data')


sr_df.to_csv('../data/service_request.csv', index = False)

### Next Steps

Proceed to the [next notebook](./2_service_request_model.ipynb), `2_service_request_model.ipynb`, for the remainder of this project. 